# Document Extraction using Multi-Modal (Text and Vision) Capabilities combining Azure AI Document Intelligence and Azure OpenAI's GPT-4o

This sample demonstrates how to build a comprehensive process to extract structured data from any document using Azure AI Document Intelligence and Azure OpenAI's GPT-4o model with vision capabilities.

![Data Extraction](../../../../images/extraction-comprehensive.png)

This is achieved by the following process:

- Analyze a document using Azure AI Document Intelligence's `prebuilt-layout` model to extract the structure as Markdown.
- Construct a system prompt that defines the instruction for extracting structured data from documents.
- Construct a user prompt that includes the specific extraction instruction for the type of document, the text content, and each document page as a base64 encoded image.
- Use the Azure OpenAI chat completions API with the GPT-4o model to generate a structured output from the content.

## Objectives

By the end of this sample, you will have learned how to:

- Convert a document to Markdown format using Azure AI Document Intelligence.
- Convert a document into a set of base64 encoded images for processing by GPT-4o.
- Use prompt engineering techniques to instruct GPT-4o to extract structured data from a type of document.
- Use the [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure) to extract structured data from the document page images using Azure OpenAI's GPT-4o model.
- Use the analysis result from Azure AI Document Intelligence to determine the confidence of the extracted structured output.
- Use the [logprobs](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#request-body:~:text=False-,logprobs,-integer) parameter in an OpenAI request to determine the confidence of the extracted structured output.

## Useful Tips

- Combine this technique with a [page classification](../../classification/README.md) approach to reduce the number of pages to extract from to only those that match your criteria for extraction.

## Setup

### Import modules

This sample takes advantage of the following .NET dependencies:

- **pdf2image-dotnet** for converting a PDF file into a set of images per page.
- **Azure.AI.DocumentIntelligence** to interface with the Azure AI Document Intelligence API for analyzing documents.
- **Azure.AI.OpenAI** to interface with the Azure OpenAI chat completions API to generate structured extraction outputs using the GPT-4o model.
- **Azure.Identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local components are also used:

- [**VehicleInsurancePolicy**](../../modules/samples/models/VehicleInsurancePolicy.csx) to provide the expected structured output JSON schema for vehicle insurance policy documents.
- [**OpenAIStructuredOutputsHelpers**](../../modules/samples/helpers/OpenAIStructuredOutputsHelpers.csx) to generate structured outputs from the OpenAI API and provide parsing functions for the response output.
- [**AccuracyEvaluator**](../../modules/samples/evaluation/AccuracyEvaluator.csx) to evaluate the output of the extraction process with expected results.
- [**DocumentIntelligenceConfidence**](../../modules/samples/confidence/DocumentIntelligenceConfidence.csx) to calculate the confidence of the extraction process based on the analysis result from the Azure AI Document Intelligence API request.
- [**OpenAIConfidence**](../../modules/samples/confidence/OpenAIConfidence.csx) to calculate the confidence of the extraction process based on the `logprobs` response from the OpenAI API request using the `Microsoft.ML.Tokenizers` and `Microsoft.ML.Tokenizers.Data.O200kBase` libraries (the latter is required for tokenizers used for the GPT-4o model).
- [**DocumentProcessingResult**](../../modules/samples/models/DocumentProcessingResult.csx) to store the results of the extraction process as a file.
- [**AppSettings**](../../modules/samples/AppSettings.csx) to access environment variables from the `.env` file.

In [1]:
#r "nuget: Azure.Identity, 1.13.2"
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: Azure.AI.DocumentIntelligence, 1.0.0"
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: Microsoft.ML.Tokenizers, 1.0.2"
#r "nuget: Microsoft.ML.Tokenizers.Data.O200kBase, 1.0.2"
#r "nuget: pdf2image-dotnet, 1.0.0"

#!import ../../modules/samples/AppSettings.csx
#!import ../../modules/samples/helpers/OpenAIStructuredOutputsHelpers.csx
#!import ../../modules/samples/helpers/StopwatchContext.csx
#!import ../../modules/samples/models/VehicleInsurancePolicy.csx
#!import ../../modules/samples/models/DocumentProcessingResult.csx
#!import ../../modules/samples/evaluation/AccuracyEvaluator.csx
#!import ../../modules/samples/confidence/ConfidenceHelpers.csx
#!import ../../modules/samples/confidence/OpenAIConfidence.csx
#!import ../../modules/samples/confidence/DocumentIntelligenceConfidence.csx

using System;
using System.IO;
using System.Text.Json;
using Azure;
using Azure.Core;
using Azure.Identity;
using Azure.AI.OpenAI;
using Azure.AI.OpenAI.Chat;
using Azure.AI.DocumentIntelligence;
using OpenAI;
using OpenAI.Chat;
using DotNetEnv;
using Pdf2Image;

Installed Packages Azure.AI.DocumentIntelligence, 1.0.0 Azure.AI.OpenAI, 2.1.0 Azure.Identity, 1.13.2 DotNetEnv, 3.1.1 Microsoft.ML.Tokenizers, 1.0.2 Microsoft.ML.Tokenizers.Data.O200kBase, 1.0.2 pdf2image-dotnet, 1.0.0

### Configure the Azure services

To use Azure AI Document Intelligence and Azure OpenAI, their SDKs are used to create client instances using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with the deployed services.

In [2]:
string workingDir = Path.GetFullPath("../../../../");
AppSettings settings = new AppSettings(new Dictionary<string, string>(Env.Load(Path.Combine(workingDir, ".env"))));
string samplePath = Path.Combine(workingDir, "samples/dotnet/extraction/multimodal/");
string sampleName = "document-extraction-gpt-text-and-vision";

DefaultAzureCredential credential = new DefaultAzureCredential(
    new DefaultAzureCredentialOptions { 
        ExcludeWorkloadIdentityCredential = true,
        ExcludeAzureDeveloperCliCredential = true,
        ExcludeEnvironmentCredential = true,
        ExcludeManagedIdentityCredential = true,
        ExcludeAzurePowerShellCredential = true,
        ExcludeSharedTokenCacheCredential = true,
        ExcludeInteractiveBrowserCredential = true
    }
);

AzureOpenAIClient openaiClient = new AzureOpenAIClient(
    new Uri(settings.OpenAIEndpoint),
    credential
);

var documentIntelligenceClient = new DocumentIntelligenceClient(
    new Uri(settings.AIServicesEndpoint),
    credential
);

### Establish the expected output

To compare the accuracy of the extraction process, the expected output of the extraction process has been defined in the following code block based on each page of a [Vehicle Insurance Policy](../../../assets/vehicle_insurance/policy_1.pdf).

> **Note**: More insurance policy examples can be found in the [assets folder](../../../assets/vehicle_insurance). These examples include the PDF file and an associated JSON metadata file that provides the expected structured output. You can add your own scenarios by following the same structure.

The expected output has been defined by a human evaluating the document.

In [3]:
string path = Path.Combine(workingDir, "samples/assets/vehicle_insurance/");
string metadataFName = "policy_5.json"; // Change this to the file you want to evaluate
string metadataFPath = Path.Combine(path, metadataFName);

Dictionary<string, JsonElement> data = JsonSerializer.Deserialize<Dictionary<string, JsonElement>>(File.ReadAllText(metadataFPath));

VehicleInsurancePolicy expected = data["expected"].Deserialize<VehicleInsurancePolicy>(
    new JsonSerializerOptions(JsonSerializerDefaults.Web)
    {
        PropertyNamingPolicy = JsonNamingPolicy.SnakeCaseLower
    }
);
string pdfFName = data["fname"].GetString();
string pdfFPath = Path.Combine(path, pdfFName);

AccuracyEvaluator<VehicleInsurancePolicy> insurancePolicyEvaluator = new AccuracyEvaluator<VehicleInsurancePolicy>(matchKeys: new List<string>());

## Extract data from the document

The following code block executes the data extraction process using Azure OpenAI's GPT-4o model using vision capabilities.

It performs the following steps:

1. Get the document bytes from the provided file path. _Note: In this example, we are processing a local document, however, you can use any document storage location of your choice, such as Azure Blob Storage._
2. Use Azure AI Document Intelligence to analyze the structure of the document and convert it to Markdown format using the pre-built layout model.
3. Use pdf2image to convert the document's pages into images per page as base64 strings.
4. Using Azure OpenAI's GPT-4o model and its [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure), extract a structured data transfer object (DTO) from the content of the images.

In [4]:
AnalyzeResult result;

StopwatchContext diSw;

using (diSw = new StopwatchContext())
{
    var pollerResult = await documentIntelligenceClient.AnalyzeDocumentAsync(
        WaitUntil.Completed,
        options: new AnalyzeDocumentOptions(
            modelId: "prebuilt-layout",
            bytesSource: BinaryData.FromBytes(File.ReadAllBytes(pdfFPath)))
        {
            OutputContentFormat = DocumentContentFormat.Markdown
        }
    );

    result = pollerResult.Value;
}

string markdown = result.Content;

In [5]:
string systemPrompt = "You are an AI assistant that extracts data from documents.";

In [6]:
List<ChatMessageContentPart> userContent = new List<ChatMessageContentPart>();

In [7]:
StringBuilder userTextPromptBuilder = new StringBuilder();
userTextPromptBuilder.AppendLine("Extract the data from this insurance policy.");
userTextPromptBuilder.AppendLine("- If a value is not present, provide null.");
userTextPromptBuilder.AppendLine("- Some values must be inferred based on the rules defined in the policy.");
userTextPromptBuilder.AppendLine("- Dates should be in the format YYYY-MM-DD.");

string userTextPrompt = userTextPromptBuilder.ToString();

userContent.Add(ChatMessageContentPart.CreateTextPart(userTextPrompt));
userContent.Add(ChatMessageContentPart.CreateTextPart(markdown));

In [8]:
StopwatchContext imageSw;

using (imageSw = new StopwatchContext())
{
    var pdfBytes = File.ReadAllBytes(pdfFPath);
    var pages = await Pdf2ImageConverter.FromBytesAsync(pdfBytes);

    userContent.AddRange(from page in pages select page into pageBytes select BinaryData.FromBytes(pageBytes) into pageData select ChatMessageContentPart.CreateImagePart(pageData, "image/png"));
}

In [9]:
ParsedChatCompletion<VehicleInsurancePolicy> completion;

StopwatchContext oaiSw;

using (oaiSw = new StopwatchContext())
{
    completion = await openaiClient
        .GetChatClient(settings.GPT4OModelDeploymentName)
        .CompleteChatAsync(
            [
                new SystemChatMessage(systemPrompt),
                new UserChatMessage(userContent)
            ],
            new ChatCompletionOptions
            {   
                ResponseFormat = CreateJsonSchemaFormat<VehicleInsurancePolicy>("vehicleInsurancePolicy", jsonSchemaIsStrict: true),
                MaxOutputTokenCount = 4096,
                Temperature = 0.1f,
                TopP = 0.1f,
                IncludeLogProbabilities = true
            }
        );
}

### Understanding the Structured Outputs JSON schema

Using [`OpenAIJsonSchema.For`](../../modules/samples/helpers/OpenAIStructuredOutputsHelpers.csx), the [VehicleInsurancePolicy](../../modules/samples/models/VehicleInsurancePolicy.csx) data model is converted to a valid OpenAI supported JSON schema when applied to the `ResponseFormat` parameter of the OpenAI chat completions request.

The JSON schema is used to instruct the GPT-4o model to generate a strict output that adheres to the structure defined. The approach makes it easier for developers to manage the data structure in code, with helpful descriptions that will be included in the final JSON schema.

Demonstrated below, you can see how the Invoice data model is understood by the OpenAI request:

In [10]:
display(typeof(VehicleInsurancePolicy).ModelJsonSchema());

{
  "type": "object",
  "properties": {
    "PolicyNumber": {
      "type": "string",
      "description": "The policy number of the vehicle insurance policy."
    },
    "Cost": {
      "anyOf": [
        {
          "$ref": "#/$defs/VehicleInsuranceCostDetails"
        },
        {
          "type": "null"
        }
      ],
      "description": "The cost details of the vehicle insurance policy."
    },
    "Renewal": {
      "anyOf": [
        {
          "$ref": "#/$defs/VehicleInsuranceRenewalDetails"
        },
        {
          "type": "null"
        }
      ],
      "description": "The renewal details of the vehicle insurance policy."
    },
    "EffectiveFrom": {
      "type": "string",
      "description": "The effective date from which the vehicle insurance policy is valid."
    },
    "EffectiveTo": {
      "type": "string",
      "description": "The effective date until which the vehicle insurance policy is valid."
    },
    "LastDateToCancel": {
      "type": "string",

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the data extraction process.

This includes:

- The accuracy of the structured data extraction comparing the expected output with the output generated by Azure OpenAI's GPT-4o model.
- The confidence score of the structured data extraction based on combining the confidence scores of the Azure AI Document Intelligence layout analysis and the log probability of the output generated by Azure OpenAI's GPT-4o model.
- The execution time of the end-to-end process.
- The total number of tokens consumed by the GPT-4o model.

### Understanding Accuracy vs Confidence

When using AI to extract structured data, both confidence and accuracy are essential for different but complementary reasons.

- **Accuracy** measures how close the AI model's output is to a ground truth or expected output. It reflects how well the model's predictions align with reality.
  - Accuracy ensures consistency in the extraction process, which is crucial for downstream tasks using the data.
- **Confidence** represents the AI model's internal assessment of how certain it is about its predictions.
  - Confidence indicates that the model is certain about its predictions, which can be a useful indicator for human reviewers to step in for manual verification.

High accuracy and high confidence are ideal, but in practice, there is often a trade-off between the two. While accuracy cannot always be self-assessed, confidence scores can and should be used to prioritize manual verification of low-confidence predictions.

In [11]:
// Displays the output of the Azure AI Document Intelligence pre-built layout analysis in Markdown format.
display(markdown);

NexGen

<!-- PageHeader="Innovation drives progress" -->

Miss Laura Bennett
18 Elm Grove
Bristol
BS1 5HQ

10th May 2024

Policy number

GB34567890123

Manage your account
NexGen.com/my-account

Hello Miss Bennett,
Welcome to your new car insurance

Thanks for choosing us. Your new car insurance is effective from 12th May 2024. Please check the summary table below
and let us know if there's anything incorrect.


<table>
<tr>
<th>Details</th>
<th>Option</th>
</tr>
<tr>
<td>HONDA CIVIC 1.8L RWD (2019) IJ90MNO</td>
<td></td>
</tr>
<tr>
<td>Class Of Use</td>
<td>Social, Domestic &amp; Pleasure</td>
</tr>
<tr>
<td>Purchase Date</td>
<td>05/02/2020</td>
</tr>
<tr>
<td>Costs of insurance</td>
<td>£750.00 per year or £62.50 per month via Direct Debit</td>
</tr>
</table>


Your new documents
You'll find your new documents in the app or MyAccount.

· Schedule of insurance

· Statement of insurance

· Certificate of insurance

· Cover summary

We're here to help

If you have any questions about t

In [12]:
// Gets the parsed VehicleInsurancePolicy object from the completion response.
var insurancePolicy = completion.Parsed;

In [13]:
// Determines the accuracy of the extracted data against the expected values.
var accuracy = insurancePolicyEvaluator.Evaluate(expected, insurancePolicy);

In [14]:
// Determines the confidence of the extracted data using the OpenAI response.
var diConfidence = DocumentIntelligenceConfidence<VehicleInsurancePolicy>.EvaluateConfidence(insurancePolicy, result);
var oaiConfidence = OpenAIConfidence<VehicleInsurancePolicy>.EvaluateConfidence(insurancePolicy, completion.Origin);

var confidence = ConfidenceHelpers.MergeConfidenceValues(diConfidence, oaiConfidence);

In [15]:
// Gets the total execution time of the data extraction process.
var totalElapsed = imageSw.Elapsed + imageSw.Elapsed + oaiSw.Elapsed;

// Gets the prompt tokens and completion tokens from the completion response.
var promptTokens = completion.Usage.InputTokenCount;
var completionTokens = completion.Usage.OutputTokenCount;

In [16]:
// Save the output of the data extraction result
var extractionResult = new DataProcessingResult<VehicleInsurancePolicy>(
    insurancePolicy,
    accuracy,
    confidence,
    promptTokens,
    completionTokens,
    totalElapsed
);

var extractionResultJson = JsonSerializer.Serialize(extractionResult, new JsonSerializerOptions { WriteIndented = true });
var extractionResultFPath = Path.Combine(samplePath, $"{sampleName}.{pdfFName}.json");

await File.WriteAllTextAsync(extractionResultFPath, extractionResultJson);

In [17]:
// Display the outputs of the extraction process.
var output = new
{
    Accuracy = $"{float.Parse(accuracy["overall"].ToString()) * 100:0.00}%",
    Confidence = $"{float.Parse(confidence["_overall"].ToString()) * 100:0.00}%",
    ExecutionTime = $"{totalElapsed.TotalSeconds:0.00} seconds",
    DocumentIntelligenceExecutionTime = $"{diSw.Elapsed.TotalSeconds:0.00} seconds",
    ImagePreprocessingTime = $"{imageSw.Elapsed.TotalSeconds:0.00} seconds",
    OpenAIExecutionTime = $"{oaiSw.Elapsed.TotalSeconds:0.00} seconds",
    PromptTokens = promptTokens,
    CompletionTokens = completionTokens,
};

display(output);
display(confidence);

Accuracy,100.00%
Confidence,99.39%
ExecutionTime,51.25 seconds
DocumentIntelligenceExecutionTime,9.51 seconds
ImagePreprocessingTime,12.86 seconds
OpenAIExecutionTime,25.53 seconds
PromptTokens,16830
CompletionTokens,243


key type value PolicyNumber System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.999998256741215 value System.String GB34567890123 Cost System.Collections.Generic.Dictionary<System.String,System.Object> key value AnnualTotal key type value confidence System.Double 0.999989435724597 value System.String 750 PayableByDate key type value confidence System.Double 0.9956227126596617 value System.String 2024-05-22 Renewal System.Collections.Generic.Dictionary<System.String,System.Object> key value RenewalNotificationDate key type value confidence System.Double 0.9730026768337359 value System.String 2025-04-20 LastDateToRenew key type value confidence System.Double 0.9925082834934512 value System.String 2025-05-04 EffectiveFrom System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9999912088304166 value System.String 2024-05-12 EffectiveTo System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9999842156349384 value System.String 2025-05-11 LastDateToCancel System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9671156507019604 value System.String 2024-05-26 Policyholder System.Collections.Generic.Dictionary<System.String,System.Object> key value FirstName key type value confidence System.Double 0.9998378327683509 value System.String Laura LastName key type value confidence System.Double 0.9998550932503032 value System.String Bennett DateOfBirth key type value confidence System.Double 0.9993106101611023 value System.String 1990-09-08 Address key type value confidence System.Double 0.9993147620359385 value System.String 18 Elm Grove, Bristol, BS1 5HQ EmailAddress key type value confidence System.Double 0.9998338782873244 value System.String LBennett@me.com TotalYearsOfResidenceInUK key type value confidence System.Double 0.9513637146664983 value System.String 29 DrivingLicenseNumber key type value confidence System.Double 0.9999968944168963 value System.String BENNEL008099JJ9IT Vehicle System.Collections.Generic.Dictionary<System.String,System.Object> key value RegistrationNumber key type value confidence System.Double 0.9999986515561293 value System.String IJ90MNO Make key type value confidence System.Double 0.9669025449367642 value System.String Honda Model key type value confidence System.Double 0.9999036769341139 value System.String Civic Year key type value confidence System.Double 0.9999878860940503 value System.String 2019 Value key type value confidence System.Double 0.9999565960025336 value System.String 11000 AccidentExcess System.Collections.Generic.Dictionary<System.String,System.Object> key value Compulsory key type value confidence System.Double 0.9998988460006815 value System.String 300 Voluntary key type value confidence System.Double 0.999996587894339 value System.String 200 UnapprovedRepairPenalty key type value confidence System.Double 0.9980614660315511 value System.String 250 FireAndTheftExcess System.Collections.Generic.Dictionary<System.String,System.Object> key value Compulsory key type value confidence System.Double 0.9999784691634502 value System.String 250 Voluntary key type value confidence System.Double 0.9999967070975446 value System.String 150 UnapprovedRepairPenalty key type value confidence System.Double 0.9994486689688259 value System.String 250 _overall System.Double 0.993917512572553